# Synchronised Click Shield Generation and Acquisition
This example shows how to synchronise multiple Red Pitaya boards to simultaneously acquire 16k samples of a generated signal on multiple Red Pitaya units (fast RF inputs and outputs) using the Red Pitaya Click Shields. Red Pitaya can transmit the trigger signal through the DIO0_N and receive it on DIO0_P.

This example can be easily modified for simultaneous generation (setup the signal generation, choose a primary trigger, all secondary triggers set to EXT_NE, and finally, change the daisy trigger source to DAC).

## Required hardware
- Two or more Red Pitaya External clock devices (STEMlab 125-14 External clock, STEMlab 125-14 4-Input)
- Red Pitaya Click Shield for each unit
- UFL Cables
- SMA cables
- SMA T-connectors


## Marcos
Throughout this tutorial we will mention macros multiple times. Here is a complete list of macros that will come in handy when customising this notebook. The marcos are a part of the **rp** library.

**GENERATION**
- **Waveforms** - RP_WAVEFORM_SINE, RP_WAVEFORM_SQUARE, RP_WAVEFORM_TRIANGLE, RP_WAVEFORM_RAMP_UP, RP_WAVEFORM_RAMP_DOWN, RP_WAVEFORM_DC, RP_WAVEFORM_PWM, RP_WAVEFORM_ARBITRARY, RP_WAVEFORM_DC_NEG, RP_WAVEFORM_SWEEP
- **Generator modes** - RP_GEN_MODE_CONTINUOUS, RP_GEN_MODE_BURST
- **Sweep direction** - RP_GEN_SWEEP_DIR_NORMAL, RP_GEN_SWEEP_DIR_UP_DOWN
- **Sweep mode** - RP_GEN_SWEEP_MODE_LINEAR, RP_GEN_SWEEP_MODE_LOG
- **Generator trigger source** - RP_GEN_TRIG_SRC_INTERNAL, RP_GEN_TRIG_SRC_EXT_PE, RP_GEN_TRIG_SRC_EXT_NE
- **Generator triggers** - RP_T_CH_1, RP_T_CH_2, RP_T_CH_EXT
- **Rise and fall times** - RISE_FALL_MIN_RATIO, RISE_FALL_MAX_RATIO

**ACQUISITION**
- **Decimation** - RP_DEC_1, RP_DEC_2, RP_DEC_4, RP_DEC_8, RP_DEC_16, RP_DEC_32, RP_DEC_64, RP_DEC_128, RP_DEC_256, RP_DEC_512, RP_DEC_1024, RP_DEC_2048, RP_DEC_4096, RP_DEC_8192, RP_DEC_16384, RP_DEC_32768, RP_DEC_65536 
- **Acquisition trigger** - RP_TRIG_SRC_DISABLED, RP_TRIG_SRC_NOW, RP_TRIG_SRC_CHA_PE, RP_TRIG_SRC_CHA_NE, RP_TRIG_SRC_CHB_PE, RP_TRIG_SRC_CHB_NE, RP_TRIG_SRC_EXT_PE, RP_TRIG_SRC_EXT_NE, RP_TRIG_SRC_AWG_PE, RP_TRIG_SRC_AWG_NE
- **Acquisition trigger state** - RP_TRIG_STATE_TRIGGERED, RP_TRIG_STATE_WAITING
- **Buffer size** - ADC_BUFFER_SIZE, DAC_BUFFER_SIZE
- **Fast analog gen_channels** - RP_CH_1, RP_CH_2

**MULTICHANNEL**
- **Shared trigger source** - OUT_TR_ADC, OUT_TR_DAC

SIGNALlab 250-12 only:
- **Input coupling** - RP_DC, RP_AC
- **Generator gain** - RP_GAIN_1X, RP_GAIN_5X

STEMlab 125-14 4-Input only:
- **Fast analog channels** - RP_CH_3, RP_CH_4
- **Acquisition trigger** - RP_TRIG_SRC_CHC_PE, RP_TRIG_SRC_CHC_NE, RP_TRIG_SRC_CHD_PE, RP_TRIG_SRC_CHD_NE

## Click Shields
Red Pitaya Click Shields are extension shields for Red Pitaya that enable a better way of synchronising *clock* and *trigger* across multiple boards, as well as attaching two MikroE click boards, and powering Red Pitaya throug either USB-C or 20 V external power source. More information regarding Red Pitaya Click Shields is available [here](https://redpitaya.readthedocs.io/en/latest/developerGuide/hardware/ext_modules/click_shield.html#click-shield).

### Note
The codes for primary and secondary units differ slightly.

### Primary unit code
The primary unit shares its TRIG signal with all other units through the Click Shield. DIO0_N is used for transmitting the trigger signal, while DIO0_P is used to receive it (EXT_TRIG). The clock signal originates from the oscillator on the primary unit's Click Shield. 

Parameters:

In [ ]:
import time
import numpy as np
from matplotlib import pyplot as plt
from rp_overlay import overlay
import rp

fpga = overlay()

########! Primary unit code !#########
channel = rp.RP_CH_1        # rp.RP_CH_2
waveform = rp.RP_WAVEFORM_SINE
freq = 100000
ampl = 1.0

trig_lvl = 0.5
trig_dly = 0

dec = rp.RP_DEC_1

gen_trig_sour = rp.RP_GEN_TRIG_SRC_INTERNAL

acq_trig_sour = rp.RP_TRIG_SRC_CHA_PE

N = 16384

# Initialize the interface
rp.rp_Init()

# Reset Generation and Acquisition
rp.rp_GenReset()
rp.rp_AcqReset()

Setting up trigger synchronisation:

In [ ]:
# Enable trigger output on DIO0_N
rp.rp_SetDpinEnableTrigOutput(True)

# Choose which trigger to synchronise (rp.OUT_TR_ADC, rp.OUT_TR_DAC)
rp.rp_SetSourceTrigOutput(rp.OUT_TR_ADC)

# LED indicator
rp.rp_DpinSetState(rp.RP_LED5, rp.RP_HIGH)

Generator code:

In [ ]:
###### Generation #####
print("Gen_start")
rp.rp_GenWaveform(channel, waveform)
rp.rp_GenFreqDirect(channel, freq)
rp.rp_GenAmp(channel, ampl)

rp.rp_GenTriggerSource(channel, gen_trig_sour)
rp.rp_GenOutEnable(channel)

Acquisition:

In [ ]:
##### Acquisition #####
rp.rp_AcqSetDecimation(dec)

# Set trigger level and delay
rp.rp_AcqSetTriggerLevel(rp.RP_T_CH_1, trig_lvl)
rp.rp_AcqSetTriggerDelay(trig_dly)

# Start Acquisition
print("Acq_start")
rp.rp_AcqStart()

# Specify trigger - input 1 positive edge
rp.rp_AcqSetTriggerSrc(acq_trig_sour)

rp.rp_GenTriggerOnly(channel)       # Trigger generator

# Trigger state
while 1:
    trig_state = rp.rp_AcqGetTriggerState()[1]
    if trig_state == rp.RP_TRIG_STATE_TRIGGERED:
        break

# Fill state
print(f"Fill state: {rp.rp_AcqGetBufferFillState()[1]}")

while 1:
    if rp.rp_AcqGetBufferFillState()[1]:
        break

### Get data ###
# Volts
fbuff = rp.fBuffer(N)
res = rp.rp_AcqGetOldestDataV(rp.RP_CH_1, N, fbuff)[1]

data_V = np.zeros(N, dtype = float)

for i in range(0, N, 1):
    data_V[i] = fbuff[i]

plt.plot(data_V)
plt.show()

# Release resources
rp.rp_Release()

### Secondary unit code
The major difference between primary and secondary unit codes is that we do not need the generator but must specify the acquisition trigger as **EXT_NE**.

The clock and trigger signals pass through the UFL cables on the Click Shields, which have buffers for correcting the signal levels on the outputs. Therefore, almost no delay is present on the clock line (besides transmission delay). The units are connected in an almost star connection.

In [ ]:
import time
import numpy as np
from matplotlib import pyplot as plt
from rp_overlay import overlay
import rp

fpga = overlay()

########! Secondary unit code !#########
channel = rp.RP_CH_1        # rp.RP_CH_2
waveform = rp.RP_WAVEFORM_SINE
freq = 100000
ampl = 1.0

trig_lvl = 0.5
trig_dly = 0

dec = rp.RP_DEC_1

# Initialize the interface
rp.rp_Init()

# Reset Generation and Acquisition
rp.rp_GenReset()
rp.rp_AcqReset()

###### Enable Trigger output #####
rp.rp_SetDpinEnableTrigOutput(True)             # on DIO0_N

# Choose which trigger to synchronise (rp.OUT_TR_ADC, rp.OUT_TR_DAC)
rp.rp_SetSourceTrigOutput(rp.OUT_TR_ADC)

# LED indicator
rp.rp_DpinSetState(rp.RP_LED5, rp.RP_HIGH)

##### Acquisition #####
rp.rp_AcqSetDecimation(dec)
rp.rp_AcqSetTriggerDelay(trig_dly)

# Start Acquisition
print("Acq_start")
rp.rp_AcqStart()

#!!! Specify trigger - must be EXT_NE !!!#
rp.rp_AcqSetTriggerSrc(rp.RP_TRIG_SRC_EXT_NE)

# Trigger state
while 1:
    trig_state = rp.rp_AcqGetTriggerState()[1]
    if trig_state == rp.RP_TRIG_STATE_TRIGGERED:
        break

# Fill state
print(f"Fill state: {rp.rp_AcqGetBufferFillState()[1]}")

while 1:
    if rp.rp_AcqGetBufferFillState()[1]:
        break

### Get data ###

# Volts
fbuff = rp.fBuffer(N)
res = rp.rp_AcqGetOldestDataV(rp.RP_CH_1, N, fbuff)[1]

data_V = np.zeros(N, dtype = float)

for i in range(0, N, 1):
    data_V[i] = fbuff[i]

plt.plot(data_V)
plt.show()


# Release resources
rp.rp_Release()

### Note
There are a lot of different commands for the acquisition and generation. The list of available functions is quite an achievement to read through, so from now on, please refer to the *C and Python API section* of the [Command table](https://redpitaya.readthedocs.io/en/latest/appsFeatures/remoteControl/command_list.html#list-of-supported-scpi-api-commands) for all available commands.
